In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models
model, tokenizer = FastLanguageModel.from_pretrained( 
    model_name = "D:/Users/Groh_Justin/PycharmProjects/LLM/models/Meta-Llama-3-70B-Instruct/original", # --> unsloth/Meta-Llama-3.1-8B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4080 SUPER. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.3.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


D:\Users\Groh_Justin\.conda\envs\unsloth_env\lib\site-packages\unsloth\models\llama.py:1189: UserWarning: expandable_segments not supported on this platform (Triggered internally at ..\c10/cuda/CUDAAllocatorConfig.h:28.)
  self.register_buffer("cos_cached", emb.cos().to(dtype=dtype, device=device, non_blocking=True), persistent=False)


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset_path = "dataset_Malware/malware_analysis_dataset_2.json"  # alpaca_data_cleaned.json, malware_analysis_dataset_2.json
dataset = load_dataset("json", data_files=dataset_path, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

<a name="Train"></a>
### Train the model


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [7]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4080 SUPER. Max memory = 15.992 GB.
5.5 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 360 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.829500
2,1.753100
3,1.768900
4,1.806000
5,1.625900
6,1.503800
7,1.358900
8,1.186000
9,1.043000
10,1.004900


In [9]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

307.3379 seconds used for training.
5.12 minutes used for training.
Peak reserved memory = 9.447 GB.
Peak reserved memory for training = 3.947 GB.
Peak reserved memory % of max memory = 59.073 %.
Peak reserved memory for training % of max memory = 24.681 %.


<a name="Inference"></a>
### Inference
Let's run the model


In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Analyze metrics to detect potential malware activities.",  # instruction
            "pslist_nproc: 157, pslist_nppid: 18, pslist_avg_threads: 9.452229299363058, pslist_nprocs64bit: 156, pslist_avg_handlers: 0.0, dlllist_ndlls: 7933, dlllist_avg_dlls_per_proc: 54.33561643835616, handles_nhandles: 52439, handles_avg_handles_per_proc: 359.17123287671234, handles_nport: 0, handles_nfile: 2056, handles_nevent: 11433, handles_ndesktop: 159, handles_nkey: 4183, handles_nthread: 1755, handles_ndirectory: 345, handles_nsemaphore: 2760, handles_ntimer: 375, handles_nsection: 1109, handles_nmutant: 861, ldrmodules_not_in_load: 489, ldrmodules_not_in_init: 636, ldrmodules_not_in_mem: 489, ldrmodules_not_in_load_avg: 0.0587740384615384, ldrmodules_not_in_init_avg: 0.0764423076923077, ldrmodules_not_in_mem_avg: 0.0587740384615384, malfind_ninjections: 26, malfind_commitCharge: 939, malfind_protection: 26, malfind_uniqueInjections: 7, psxview_not_in_pslist: 9, psxview_not_in_eprocess_pool: 2, psxview_not_in_ethread_pool: 10, psxview_not_in_pspcid_list: 165, psxview_not_in_csrss_handles: 21, psxview_not_in_session: 0, psxview_not_in_deskthrd: 165, psxview_not_in_pslist_false_avg: 0.0545454545454545, psxview_not_in_eprocess_pool_false_avg: 0.0121212121212121, psxview_not_in_ethread_pool_false_avg: 0.0606060606060606, psxview_not_in_pspcid_list_false_avg: 1.0, psxview_not_in_csrss_handles_false_avg: 0.1272727272727272, psxview_not_in_session_false_avg: 0.0, psxview_not_in_deskthrd_false_avg: 1.0, modules_nmodules: 175, svcscan_nservices: 0, svcscan_kernel_drivers: 0, svcscan_fs_drivers: 0, svcscan_process_services: 0, svcscan_shared_process_services: 0, svcscan_interactive_process_services: 0, svcscan_nactive: 0, callbacks_ncallbacks: 264, callbacks_nanonymous: 223, callbacks_ngeneric: 19",
            # input
            "",  # output - leave this blank for generation!
        )
    ], return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
#outputs = model.generate(
#    **inputs, 
#    max_new_tokens=512, 
#    use_cache=True, 
#    temperature=0.7,  # Zwischen 0 und 1, höher für kreativere Antworten
#    top_p=0.9  # Top-p sampling, bei dem die Wahrscheinlichkeit der Top-N-Tokens berücksichtigt wird
#)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnalyze metrics to detect potential malware activities.\n\n### Input:\npslist_nproc: 157, pslist_nppid: 18, pslist_avg_threads: 9.452229299363058, pslist_nprocs64bit: 156, pslist_avg_handlers: 0.0, dlllist_ndlls: 7933, dlllist_avg_dlls_per_proc: 54.33561643835616, handles_nhandles: 52439, handles_avg_handles_per_proc: 359.17123287671234, handles_nport: 0, handles_nfile: 2056, handles_nevent: 11433, handles_ndesktop: 159, handles_nkey: 4183, handles_nthread: 1755, handles_ndirectory: 345, handles_nsemaphore: 2760, handles_ntimer: 375, handles_nsection: 1109, handles_nmutant: 861, ldrmodules_not_in_load: 489, ldrmodules_not_in_init: 636, ldrmodules_not_in_mem: 489, ldrmodules_not_in_load_avg: 0.0587740384615384, ldrmodules_not_in_init_avg: 0.0764423076923077, ldrmodules_not_in_mem_avg: 0.05877

`TextStreamer` for continuous inference 

In [16]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Analyze metrics to detect potential malware activities.", # instruction
        "pslist_nproc: 132, pslist_nppid: 17, pslist_avg_threads: 8.545454545454545, pslist_nprocs64bit: 130, pslist_avg_handlers: 0.0, dlllist_ndlls: 6163, dlllist_avg_dlls_per_proc: 49.304, handles_nhandles: 41957, handles_avg_handles_per_proc: 332.9920634920635, handles_nport: 0, handles_nfile: 1760, handles_nevent: 9393, handles_ndesktop: 127, handles_nkey: 3167, handles_nthread: 1312, handles_ndirectory: 294, handles_nsemaphore: 1891, handles_ntimer: 323, handles_nsection: 929, handles_nmutant: 678, ldrmodules_not_in_load: 416, ldrmodules_not_in_init: 543, ldrmodules_not_in_mem: 416, ldrmodules_not_in_load_avg: 0.0632603406326034, ldrmodules_not_in_init_avg: 0.0825729927007299, ldrmodules_not_in_mem_avg: 0.0632603406326034, malfind_ninjections: 22, malfind_commitCharge: 1347, malfind_protection: 22, malfind_uniqueInjections: 6, psxview_not_in_pslist: 3, psxview_not_in_eprocess_pool: 0, psxview_not_in_ethread_pool: 6, psxview_not_in_pspcid_list: 134, psxview_not_in_csrss_handles: 13, psxview_not_in_session: 0, psxview_not_in_deskthrd: 134, psxview_not_in_pslist_false_avg: 0.0223880597014925, psxview_not_in_eprocess_pool_false_avg: 0.0, psxview_not_in_ethread_pool_false_avg: 0.044776119402985, psxview_not_in_pspcid_list_false_avg: 1.0, psxview_not_in_csrss_handles_false_avg: 0.0970149253731343, psxview_not_in_session_false_avg: 0.0, psxview_not_in_deskthrd_false_avg: 1.0, modules_nmodules: 175, svcscan_nservices: 1500, svcscan_kernel_drivers: 768, svcscan_fs_drivers: 87, svcscan_process_services: 156, svcscan_shared_process_services: 2, svcscan_interactive_process_services: 2, svcscan_nactive: 388, callbacks_ncallbacks: 263, callbacks_nanonymous: 223, callbacks_ngeneric: 19", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

#_ = model.generate(
#    **inputs, 
#    streamer = text_streamer,
#    max_new_tokens=512, 
#    use_cache=True, 
#    temperature=0.7,  # Zwischen 0 und 1, höher für kreativere Antworten
#    top_p=0.9  # Top-p sampling, bei dem die Wahrscheinlichkeit der Top-N-Tokens berücksichtigt wird
#)

_ = model.generate(
    **inputs, 
    streamer = text_streamer,
    max_new_tokens=512, 
    use_cache=True, 
)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze metrics to detect potential malware activities.

### Input:
pslist_nproc: 132, pslist_nppid: 17, pslist_avg_threads: 8.545454545454545, pslist_nprocs64bit: 130, pslist_avg_handlers: 0.0, dlllist_ndlls: 6163, dlllist_avg_dlls_per_proc: 49.304, handles_nhandles: 41957, handles_avg_handles_per_proc: 332.9920634920635, handles_nport: 0, handles_nfile: 1760, handles_nevent: 9393, handles_ndesktop: 127, handles_nkey: 3167, handles_nthread: 1312, handles_ndirectory: 294, handles_nsemaphore: 1891, handles_ntimer: 323, handles_nsection: 929, handles_nmutant: 678, ldrmodules_not_in_load: 416, ldrmodules_not_in_init: 543, ldrmodules_not_in_mem: 416, ldrmodules_not_in_load_avg: 0.0632603406326034, ldrmodules_not_in_init_avg: 0.0825729927007299, ldrmodules_not_in_mem_avg: 0.0632603406326034, malfind_n

<a name="Save"></a>
### Saving, loading finetuned models
**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

### Saving to float16 for VLLM

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if True: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )